### COVID-19 ETL task

#### 1. Python: Download a timeseries of daily deaths per country

In [2]:
import pandas as pd   #  python 3.6+, pd 0.25+
from datetime import datetime, date
from sqlalchemy import create_engine  #sqlite

In [3]:
COVID_DATA = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/' \
                 'csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'

In [30]:
df = pd.read_csv(COVID_DATA, skipinitialspace=True).fillna('')
print(df.shape, 'n/a:', pd.isna(df).sum().sum())
df.head()

(259, 78) n/a: 0


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20
0,,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,4,4,4,4,4,4,4,6,6,7
1,,Albania,41.1533,20.1683,0,0,0,0,0,0,...,6,8,10,10,11,15,15,16,17,20
2,,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,25,26,29,31,35,44,58,86,105,130
3,,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,3,3,3,6,8,12,14,15,16,17
4,,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,0,0,0,2,2,2,2,2,2,2


In [31]:
# print( df.loc[df['Country/Region']=='France', 'Province/State'] )
df['Country/Region'].value_counts()[:4]

China             33
Canada            15
United Kingdom    11
France            10
Name: Country/Region, dtype: int64

Some countries have colonies/territories or shown by state/region.  
Sum that data by country (! check your policy ~French Guiana, Taiwan)
Dropping "Province" and geo data.  

In [32]:
df = df.drop(columns=['Province/State', 'Lat', 'Long'])
df = df.rename(columns={'Country/Region': 'country'})
df = df.groupby('country', as_index=False).sum()
print(df.shape)

(181, 75)


#### 2. Python: Convert the table so that each country and each day is a separate row

In [33]:
df1 = df.melt(id_vars=['country'], value_vars=df.columns[1:], var_name='date', value_name='cum_deaths')
df1['date'] = pd.to_datetime(df1['date'])
df1.reindex()
df1.sample(5)

,country,date,cum_deaths
12416,Malta,2020-03-30,0
9820,Denmark,2020-03-16,3
10061,Maldives,2020-03-17,0
4247,Italy,2020-02-14,0
1578,Papua New Guinea,2020-01-30,0


#### 3. Python: Provide code to upload the table from step 3 into an SQL table named deaths_total

In [42]:
# Using sqlite file in the same folder, no credentials
engine = create_engine('sqlite:///covid_d.db')
conn = engine.connect()

In [35]:
# ! Attention !  Replacing existing table (not append) 
df1.to_sql('deaths_total', con=conn, index=False, if_exists='replace')
#df1.to_sql('deaths_total', con=engine, if_exists='append')

In [36]:
# just checking
conn.execute("SELECT date, cum_deaths FROM deaths_total WHERE country='Australia' ORDER BY date DESC LIMIT 3").fetchall()

[('2020-04-04 00:00:00.000000', 30),
 ('2020-04-03 00:00:00.000000', 28),
 ('2020-04-02 00:00:00.000000', 24)]

#### 4. Python: From the data in step 2, calculate the daily change in deaths for each country 

In [37]:
df2 = df1.copy()

# Daily deaths go to the first day. last day N/A = 0 

for c in df2.country.unique():
    c_ind = df2.country==c
    df2.loc[c_ind, 'day_deaths'] = ( df2.loc[c_ind, 'cum_deaths'].shift(-1) - df2.loc[c_ind, 'cum_deaths'] )

df2.day_deaths = df2.day_deaths.fillna(0).astype('int') 
df2.sample(5)

,country,date,cum_deaths,day_deaths
7110,Ecuador,2020-03-01,0,0
12610,New Zealand,2020-03-31,1,0
9805,Cameroon,2020-03-16,0,0
8830,Saint Vincent and the Grenadines,2020-03-10,0,0
3287,Cabo Verde,2020-02-09,0,0


#### 5. Python: Provide code to upload the table from step 4 into an SQL table named deaths_change_python

In [38]:
# ! Attention !  Replacing existing table (not append)
df2.to_sql('deaths_change_python', con=conn, index=False, if_exists='replace')

#### 6. SQL: Provide SQL code to calculate the daily change for each country using only the data from deaths_total and save it into an SQL table named deaths_change_sql

In [39]:
#  ! Attention !  Deleting existing table 
conn.execute('DROP TABLE IF EXISTS deaths_change_sql;')

In [40]:
conn.execute(''' 
    CREATE TABLE deaths_change_sql AS
        SELECT country, date, cum_deaths,
               COALESCE( LEAD(cum_deaths, 1) OVER(PARTITION BY country) - cum_deaths, 0) AS day_deaths 
        FROM deaths_total  
        ORDER BY date, country;
''')

In [44]:
engine.table_names()

['deaths_change_python',
 'deaths_change_sql',
 'deaths_total',
 'sqlite_sequence']

In [45]:
conn.close()  # conn.closed

##### misc

In [ ]:
# to create executable python code
# jupyter nbconvert --to script covid_ETL.ipynb